In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
from torchsummary import summary


In [ ]:
from google.colab import drive
drive.mount('/content/drive' , force_remount=True)

Mounted at /content/drive


In [ ]:
class CNN_classifier(nn.Module):
    def __init__(self):
        super().__init__() # To avoid referring to the base class explicitly 
        # (1) conv.layer 1
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3)
        self.conv1_bn = nn.BatchNorm2d(32)
        # (2) conv.layer 2
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3)
        self.conv2_bn = nn.BatchNorm2d(64)
        # (3) conv.layer 3
        self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3)
        self.conv3_bn = nn.BatchNorm2d(128)
        # (4) conv.layer 4
        self.conv4 = nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 5)
        self.conv4_bn = nn.BatchNorm2d(256)
        # (5) conv.layer 5
        self.conv5 = nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 5)
        self.conv5_bn = nn.BatchNorm2d(512)
        
        # (1) fully connected layer 1
        self.fc1 = nn.Linear(in_features=4*4*512, out_features=4000)
        self.fc1_bn = nn.BatchNorm1d(4000)
        self.dropout1=nn.Dropout(0.25)
        # (2) fully connected layer 2
        self.fc2 = nn.Linear(in_features=4000, out_features=400)
        self.fc2_bn = nn.BatchNorm1d(400)
        self.dropout2=nn.Dropout(0.25)
        
        # output layer
        self.out = nn.Linear(in_features=400, out_features=10)
        
    def forward(self, t):
            
        #conv. layers
        t = self.conv1(t)
        t = F.leaky_relu(t, 0.1)
        t = self.conv1_bn(t)

        t = F.max_pool2d(t, kernel_size=2, stride=2)
      
        t = self.conv2(t)
        t = F.leaky_relu(t, 0.1)
        t = self.conv2_bn(t)

        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = self.conv3(t)
        t = F.leaky_relu(t, 0.1)
        t = self.conv3_bn(t)

        t = F.max_pool2d(t, kernel_size=2, stride=2)
      
        t = self.conv4(t)
        t = F.leaky_relu(t, 0.05)
        t = self.conv4_bn(t)

        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = self.conv5(t)
        t = F.leaky_relu(t, 0.05)
        t = self.conv5_bn(t)

        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = t.reshape(-1, 4*4*512)
        t = self.fc1(t)
        t = self.dropout1(t)
        t = F.leaky_relu(t, 0.05)
        t = self.fc1_bn(t)

       
        t = self.fc2(t)
        t = self.dropout2(t)
        t = F.leaky_relu(t, 0.05)
        t = self.fc2_bn(t)

        #output layer
        t = self.out(t)
        t = F.softmax(t, dim=1)
        return t

In [ ]:
device='cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
model = CNN_classifier().to(device)

In [ ]:
#train_dir="/content/drive/MyDrive/kaggledata/train_main/train_aug2/train/"
#test_dir="/content/drive/MyDrive/kaggledata/test/"

train_dir="/content/drive/MyDrive/Datasets/GNR/train_aug"
test_dir="/content/drive/MyDrive/Datasets/GNR/test"
torch.manual_seed(108)
train = datasets.ImageFolder(train_dir, transform=transforms.Compose([transforms.ToTensor()]), target_transform=None, is_valid_file=None)
print(len(train))
train_set, val_set = torch.utils.data.random_split(train, [5000,500]) 
train_data = torch.utils.data.DataLoader(train_set, batch_size = 10, num_workers=2, shuffle = True)
val_data = torch.utils.data.DataLoader(val_set, batch_size = 5, num_workers=2, shuffle = True)

5500


In [ ]:
test_set = datasets.ImageFolder(root=test_dir, transform=transforms.Compose([transforms.ToTensor()]), target_transform=None, is_valid_file=None)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=10, shuffle=False)

In [ ]:
train_data

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum = 0.5)
acc0=0
acc1=0
test_accuracy =0
for epoch in range(30): #50 no. of epochs
    total_loss=0
    total_preds = 0
    val_total_preds = 0
    count = 0

    for batch in train_data:
        model.train()
        images, labels = batch
        optimizer.zero_grad()                                                  # assining the gradients value 0
        preds = model(images.to(device))                                         # Pass Batch
        loss = F.cross_entropy(preds, labels.to(device))                           # Calculate Loss
        loss.backward()                                                        # Calculate Gradients
        optimizer.step()                                                       # Update Weights
        total_loss += loss.item()                                              # Calculating total loss 
        total_preds += preds.argmax(dim=1).eq(labels.to(device)).sum().item()      # Calculating total no. of correct preditions 
        count =count+1  
    train_acc= (((float) ((total_preds)))/5000) *100
    val_acc = val_total_preds/5
    print("Epoch: {}, Training Loss: {:0.5f}, Training Accuracy: {:0.3f}% Validation Accuracy: {:0.3f}%".format(epoch, total_loss, train_acc, test_accuracy))
    

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 0, Training Loss: 852.07594, Training Accuracy: 81.880% Validation Accuracy: 0.000%
Epoch: 1, Training Loss: 775.72709, Training Accuracy: 93.780% Validation Accuracy: 0.000%
Epoch: 2, Training Loss: 757.63276, Training Accuracy: 96.660% Validation Accuracy: 0.000%
Epoch: 3, Training Loss: 746.63244, Training Accuracy: 98.340% Validation Accuracy: 0.000%
Epoch: 4, Training Loss: 741.01235, Training Accuracy: 99.120% Validation Accuracy: 0.000%
Epoch: 5, Training Loss: 740.68055, Training Accuracy: 98.920% Validation Accuracy: 0.000%
Epoch: 6, Training Loss: 738.55102, Training Accuracy: 99.300% Validation Accuracy: 0.000%
Epoch: 7, Training Loss: 736.46721, Training Accuracy: 99.460% Validation Accuracy: 0.000%
Epoch: 8, Training Loss: 735.73957, Training Accuracy: 99.540% Validation Accuracy: 0.000%
Epoch: 9, Training Loss: 735.82083, Training Accuracy: 99.480% Validation Accuracy: 0.000%
Epoch: 10, Training Loss: 734.75720, Training Accuracy: 99.760% Validation Accuracy: 0.000

In [ ]:
#torch.save(model, "/content/drive/MyDrive/kaggledata/model_sm.pkl")
torch.save(model.state_dict(), "/content/drive/MyDrive/Datasets/GNR/MS2")

In [ ]:
test_set = datasets.ImageFolder(root=test_dir, transform=transforms.Compose([transforms.ToTensor()]), target_transform=None, is_valid_file=None)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=100, shuffle=False)

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/Datasets/GNR/MS2"))
for i in test_loader:
    my_image,label = i
    prediction = model(my_image.to(device))
    print(prediction.argmax(dim = 1))
result = prediction.argmax(dim = 1)
y_test=[4., 9., 2., 4., 1., 0., 9., 3., 1., 1., 0., 0., 4., 5., 0., 8., 7., 7.,
        7., 5., 4., 1., 5., 3., 2., 5., 6., 6., 4., 2., 0., 0., 2., 7., 1., 3.,
        2., 1., 6., 8., 9., 9., 0., 3., 6., 8., 9., 4., 6., 5., 7., 7., 0., 6.,
        8., 3., 9., 2., 6., 1., 9., 2., 4., 1., 0., 3., 7., 8., 3., 9., 4., 6.,
        8., 1., 7., 6., 6., 7., 2., 5., 1., 3., 1., 7., 6., 9., 7., 2., 4., 6.,
        9., 1., 5., 8., 1., 6., 4., 3., 9., 7.]
y_test = torch.tensor(np.array(y_test))
print(torch.eq(y_test.to(device), result).sum())

tensor([4, 9, 2, 4, 0, 0, 9, 3, 0, 1, 6, 0, 4, 5, 7, 8, 5, 4, 7, 7, 4, 3, 5, 3,
        2, 5, 6, 6, 4, 2, 8, 0, 2, 7, 6, 3, 2, 1, 5, 8, 9, 9, 0, 3, 6, 3, 9, 4,
        0, 8, 7, 7, 0, 5, 0, 3, 9, 2, 6, 6, 9, 5, 8, 3, 6, 3, 7, 8, 3, 9, 4, 5,
        8, 6, 7, 2, 5, 0, 2, 8, 6, 3, 0, 7, 5, 9, 7, 2, 4, 2, 5, 1, 5, 8, 1, 6,
        4, 4, 7, 7], device='cuda:0')
tensor(66, device='cuda:0')


In [ ]:
y_pred = np.array(result.cpu()).astype(int)
y_pred= y_pred+1
print(y_pred)
a = {'ImageID': range(101, 201), 'LabelID': y_pred}
b = pd.DataFrame(a)
print(b)
b.to_csv('19D070070.csv', index = False)

[ 5 10  3  5  1  1 10  4  1  2  7  1  5  6  8  9  6  5  8  8  5  4  6  4
  3  6  7  7  5  3  9  1  3  8  7  4  3  2  6  9 10 10  1  4  7  4 10  5
  1  9  8  8  1  6  1  4 10  3  7  7 10  6  9  4  7  4  8  9  4 10  5  6
  9  7  8  3  6  1  3  9  7  4  1  8  6 10  8  3  5  3  6  2  6  9  2  7
  5  5  8  8]
    ImageID  LabelID
0       101        5
1       102       10
2       103        3
3       104        5
4       105        1
..      ...      ...
95      196        7
96      197        5
97      198        5
98      199        8
99      200        8

[100 rows x 2 columns]
